In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("akilesh253/sugarcane-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sugarcane-plant-diseases-dataset


In [2]:
import os, shutil, random, zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2, EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report

base_dir = "/kaggle/input/sugarcane-plant-diseases-dataset/Sugarcane_leafs"  
balanced_dir = "/kaggle/working/sugarcane_balanced"

img_size = (224,224)
batch_size = 32

2025-09-03 16:24:40.617571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756916680.789638      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756916680.843116      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
if os.path.exists(balanced_dir):
    shutil.rmtree(balanced_dir)
os.makedirs(balanced_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Count images per class
orig_counts = {cls: len(os.listdir(os.path.join(base_dir, cls))) for cls in os.listdir(base_dir)}
max_count = max(orig_counts.values())
print("📊 Original class distribution:", orig_counts)

for cls, count in orig_counts.items():
    src = os.path.join(base_dir, cls)
    dst = os.path.join(balanced_dir, cls)
    os.makedirs(dst, exist_ok=True)

    files = os.listdir(src)

    # copy originals
    for f in files:
        shutil.copy(os.path.join(src, f), os.path.join(dst, f))

    # augment if needed
    if count < max_count:
        i = 0
        while count + i < max_count:
            img_path = random.choice(files)
            img = load_img(os.path.join(src, img_path), target_size=img_size)
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)

            for batch in datagen.flow(x, batch_size=1,
                                      save_to_dir=dst,
                                      save_prefix="aug",
                                      save_format="jpg"):
                i += 1
                if count + i >= max_count:
                    break

# check new counts
new_counts = {cls: len(os.listdir(os.path.join(balanced_dir, cls))) for cls in os.listdir(balanced_dir)}
print("✅ Balanced class distribution:", new_counts)


📊 Original class distribution: {'Yellow': 3030, 'Mosaic': 2772, 'BacterialBlights': 4800, 'Healthy': 3132, 'RedRot': 3108, 'Rust': 3084}
✅ Balanced class distribution: {'RedRot': 4683, 'Rust': 4670, 'Healthy': 4665, 'Yellow': 4631, 'Mosaic': 4609, 'BacterialBlights': 4800}


In [4]:
import os, shutil, random

train_dir = "/kaggle/working/sugarcane_balanced/train"
val_dir   = "/kaggle/working/sugarcane_balanced/validation"

# clean old dirs
if os.path.exists(train_dir): shutil.rmtree(train_dir)
if os.path.exists(val_dir): shutil.rmtree(val_dir)
os.makedirs(train_dir), os.makedirs(val_dir)

for cls in os.listdir(balanced_dir):
    cls_path = os.path.join(balanced_dir, cls)
    if not os.path.isdir(cls_path):  # skip any accidental non-folder items
        continue
    
    os.makedirs(os.path.join(train_dir, cls))
    os.makedirs(os.path.join(val_dir, cls))

    #  only keep files (ignore directories)
    files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
    random.shuffle(files)
    split = int(0.8 * len(files))

    for f in files[:split]:
        shutil.copy(os.path.join(cls_path, f), os.path.join(train_dir, cls, f))
    for f in files[split:]:
        shutil.copy(os.path.join(cls_path, f), os.path.join(val_dir, cls, f))

print("✅ Train/Val split done")

✅ Train/Val split done


In [5]:
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical", shuffle=False
)

class_names = list(train_gen.class_indices.keys())
print("✅ Classes:", class_names)


Found 22445 images belonging to 8 classes.
Found 5613 images belonging to 8 classes.
✅ Classes: ['BacterialBlights', 'Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow', 'train', 'validation']


In [6]:
#CNN
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'), MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'), MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

cnn_model.compile(optimizer=Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"])
early_stop = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)

history_cnn = cnn_model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[early_stop], verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1756917157.161845      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1756917161.006755     115 service.cc:148] XLA service 0x7b14980062f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756917161.007722     115 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756917161.317347     115 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/702 ━━━━━━━━━━━━━━━━━━━━ 44s 63ms/step - accuracy: 0.1337 - loss: 2.5425  

I0000 00:00:1756917165.325235     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


702/702 ━━━━━━━━━━━━━━━━━━━━ 76s 99ms/step - accuracy: 0.4445 - loss: 1.4391 - val_accuracy: 0.7481 - val_loss: 0.7029
Epoch 2/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 66s 93ms/step - accuracy: 0.7178 - loss: 0.8147 - val_accuracy: 0.7848 - val_loss: 0.5821
Epoch 3/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 64s 91ms/step - accuracy: 0.7568 - loss: 0.6571 - val_accuracy: 0.8087 - val_loss: 0.5305
Epoch 4/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 62s 89ms/step - accuracy: 0.8051 - loss: 0.5287 - val_accuracy: 0.8341 - val_loss: 0.4596
Epoch 5/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.8295 - loss: 0.4546 - val_accuracy: 0.8373 - val_loss: 0.4539
Epoch 6/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.8519 - loss: 0.3846 - val_accuracy: 0.8443 - val_loss: 0.4359
Epoch 7/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 63s 89ms/step - accuracy: 0.8747 - loss: 0.3326 - val_accuracy: 0.8532 - val_loss: 0.4412
Epoch 8/50
702/702 ━━━━━━━━━━━━━━━━━━━━ 62s 88ms/step - accuracy: 0.8880 - loss: 0.2944 - val_accurac

In [7]:
#MobileNetV2
base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False

mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

mobilenet_model.compile(optimizer=Adam(5e-4), loss="categorical_crossentropy", metrics=["accuracy"])
mobilenet_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stop], verbose=1)

base_model.trainable = True
mobilenet_model.compile(optimizer=Adam(1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
history_mobilenet = mobilenet_model.fit(train_gen, validation_data=val_gen, epochs=40, callbacks=[early_stop], verbose=1)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 82s 102ms/step - accuracy: 0.7111 - loss: 0.8281 - val_accuracy: 0.8742 - val_loss: 0.3484
Epoch 2/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.8610 - loss: 0.3783 - val_accuracy: 0.8890 - val_loss: 0.3086
Epoch 3/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 64s 91ms/step - accuracy: 0.8874 - loss: 0.3084 - val_accuracy: 0.8963 - val_loss: 0.2828
Epoch 4/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 64s 90ms/step - accuracy: 0.8967 - loss: 0.2802 - val_accuracy: 0.9020 - val_loss: 0.2662
Epoch 5/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 63s 90ms/step - accuracy: 0.9066 - loss: 0.2553 - val_accuracy: 0.9017 - val_loss: 0.2624
Epoch 6/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.9159 - loss: 0.2269 - val_accuracy: 0.9109 - val_loss: 0.2417
Epoch 7/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 65s 93ms/step - accuracy: 0.9206 - loss: 0.2126 - val_accuracy: 0.9147 - val_loss: 0.2289
Epoch 8/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 

E0000 00:00:1756919863.602341     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756919863.799434     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


415/702 ━━━━━━━━━━━━━━━━━━━━ 23s 80ms/step - accuracy: 0.6533 - loss: 1.3904

E0000 00:00:1756919914.546160     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756919914.742424     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


702/702 ━━━━━━━━━━━━━━━━━━━━ 146s 131ms/step - accuracy: 0.6960 - loss: 1.1630 - val_accuracy: 0.8587 - val_loss: 0.4618
Epoch 2/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 69s 98ms/step - accuracy: 0.8582 - loss: 0.3918 - val_accuracy: 0.8610 - val_loss: 0.4355
Epoch 3/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 70s 100ms/step - accuracy: 0.8965 - loss: 0.2758 - val_accuracy: 0.8911 - val_loss: 0.3190
Epoch 4/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 68s 97ms/step - accuracy: 0.9136 - loss: 0.2378 - val_accuracy: 0.9164 - val_loss: 0.2370
Epoch 5/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 69s 98ms/step - accuracy: 0.9288 - loss: 0.1940 - val_accuracy: 0.9266 - val_loss: 0.1936
Epoch 6/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 68s 97ms/step - accuracy: 0.9461 - loss: 0.1490 - val_accuracy: 0.9359 - val_loss: 0.1748
Epoch 7/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 68s 96ms/step - accuracy: 0.9555 - loss: 0.1208 - val_accuracy: 0.9446 - val_loss: 0.1523
Epoch 8/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 68s 97ms/step - accuracy: 0.9612 - loss: 0.1050 - val_accu

In [8]:
#EfficientNetB3
eff_base = EfficientNetB3(input_shape=(224,224,3), include_top=False, weights='imagenet')
eff_base.trainable = False

x = eff_base.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
output = Dense(len(class_names), activation="softmax")(x)
eff_model = Model(inputs=eff_base.input, outputs=output)

eff_model.compile(optimizer=Adam(1e-4), loss="categorical_crossentropy", metrics=["accuracy"])
eff_model.fit(train_gen, validation_data=val_gen, epochs=10, callbacks=[early_stop], verbose=1)

eff_base.trainable = True
eff_model.compile(optimizer=Adam(1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
history_eff = eff_model.fit(train_gen, validation_data=val_gen, epochs=40, callbacks=[early_stop], verbose=1)


43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 137s 138ms/step - accuracy: 0.1865 - loss: 1.8499 - val_accuracy: 0.2672 - val_loss: 1.7308
Epoch 2/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 69s 98ms/step - accuracy: 0.2334 - loss: 1.7468 - val_accuracy: 0.2811 - val_loss: 1.6977
Epoch 3/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.2431 - loss: 1.7250 - val_accuracy: 0.2443 - val_loss: 1.6814
Epoch 4/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.2553 - loss: 1.7140 - val_accuracy: 0.2856 - val_loss: 1.6681
Epoch 5/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.2551 - loss: 1.6982 - val_accuracy: 0.2920 - val_loss: 1.6618
Epoch 6/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 67s 95ms/step - accuracy: 0.2577 - loss: 1.7012 - val_accuracy: 0.2950 - val_loss: 1.6588
Epoch 7/10
702/702 ━━━━━━━━━━━━━━━━━━━━ 70s 100ms/step - accuracy: 0.2578 - loss: 1.6978 - val_accuracy: 0.2968 - val_loss: 1.6556
Epoch 8/10
702/702 ━━━━━━━━━━━━━━━━━

E0000 00:00:1756923543.959982     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923544.160091     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923544.707241     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923544.926107     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923545.329417     115 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

684/702 ━━━━━━━━━━━━━━━━━━━━ 3s 209ms/step - accuracy: 0.2715 - loss: 2.8033

E0000 00:00:1756923734.092396     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923734.285414     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923734.759725     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923734.976467     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756923735.355173     112 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

702/702 ━━━━━━━━━━━━━━━━━━━━ 353s 313ms/step - accuracy: 0.2774 - loss: 2.7740 - val_accuracy: 0.7304 - val_loss: 0.9277
Epoch 2/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 160s 228ms/step - accuracy: 0.7595 - loss: 0.7608 - val_accuracy: 0.8286 - val_loss: 0.5351
Epoch 3/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 160s 228ms/step - accuracy: 0.8606 - loss: 0.4287 - val_accuracy: 0.8381 - val_loss: 0.4625
Epoch 4/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 161s 229ms/step - accuracy: 0.9002 - loss: 0.2965 - val_accuracy: 0.9307 - val_loss: 0.1955
Epoch 5/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 161s 229ms/step - accuracy: 0.9245 - loss: 0.2232 - val_accuracy: 0.9309 - val_loss: 0.2055
Epoch 6/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 161s 229ms/step - accuracy: 0.9376 - loss: 0.1794 - val_accuracy: 0.9385 - val_loss: 0.1672
Epoch 7/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 161s 229ms/step - accuracy: 0.9515 - loss: 0.1445 - val_accuracy: 0.9524 - val_loss: 0.1258
Epoch 8/40
702/702 ━━━━━━━━━━━━━━━━━━━━ 161s 228ms/step - accuracy: 0.9566 - loss: 0.12

In [9]:
display_names = []
for cls in val_gen.class_indices.keys():
    if cls.lower() == "healthy":
        display_names.append("Healthy")
    else:
        display_names.append(f"Infected - {cls}")

print("✅ Display Names:", display_names)


✅ Display Names: ['Infected - BacterialBlights', 'Healthy', 'Infected - Mosaic', 'Infected - RedRot', 'Infected - Rust', 'Infected - Yellow', 'Infected - train', 'Infected - validation']


In [10]:
results = {}

def evaluate_and_save(model, history, name):
    preds = model.predict(val_gen, verbose=0)
    y_true = val_gen.classes
    y_pred = np.argmax(preds, axis=1)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=display_names, yticklabels=display_names)
    plt.title(f"{name} Confusion Matrix")
    plt.savefig(f"{name}_confusion.png")
    plt.close()

    # Classification Report
    report = classification_report(
        y_true, y_pred, 
        target_names=display_names[:len(np.unique(y_true))],  # ensures match
        digits=4
    )
    with open(f"{name}_report.txt", "w") as f:
        f.write(report)

    # Training Curves
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label="Train Acc")
    plt.plot(history.history['val_accuracy'], label="Val Acc")
    plt.title(f"{name} Accuracy")
    plt.xlabel("Epochs"); plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label="Train Loss")
    plt.plot(history.history['val_loss'], label="Val Loss")
    plt.title(f"{name} Loss")
    plt.xlabel("Epochs"); plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.savefig(f"{name}_curves.png")
    plt.close()

    # Save model
    model.save(f"{name}.h5")

    # Save best val acc
    acc = max(history.history['val_accuracy'])
    results[name] = acc
    print(f"✅ {name} evaluation saved (Best Val Acc: {acc:.4f})")


# =============================
# EVALUATE ALL MODELS
# =============================
evaluate_and_save(cnn_model, history_cnn, "CNN")
evaluate_and_save(mobilenet_model, history_mobilenet, "MobileNetV2")
evaluate_and_save(eff_model, history_eff, "EfficientNetB3")

✅ CNN evaluation saved (Best Val Acc: 0.8737)
✅ MobileNetV2 evaluation saved (Best Val Acc: 0.9669)
✅ EfficientNetB3 evaluation saved (Best Val Acc: 0.9745)


In [11]:
plt.figure(figsize=(7,5))
plt.bar(results.keys(), results.values(), color=["skyblue","lightgreen","salmon"])
plt.ylabel("Validation Accuracy")
plt.title("Model Accuracy Comparison")
for i, acc in enumerate(results.values()):
    plt.text(i, acc+0.005, f"{acc:.2%}", ha="center")
plt.savefig("model_comparison.png")
plt.close()

In [12]:
def compare_predictions(models, names, n=6):
    sample_paths = random.sample(val_gen.filepaths, n)
    plt.figure(figsize=(4*n, 12))
    for row, (model, name) in enumerate(zip(models, names)):
        for col, path in enumerate(sample_paths):
            img = load_img(path, target_size=img_size)
            arr = np.expand_dims(img_to_array(img)/255.0, axis=0)
            pred = np.argmax(model.predict(arr, verbose=0), axis=1)[0]

            pred_label = display_names[pred]
            true_label = path.split("/")[-2]
            true_label_disp = "Healthy" if true_label.lower()=="healthy" else f"Infected - {true_label}"

            ax = plt.subplot(len(models), n, row*n + col + 1)
            plt.imshow(img); plt.axis("off")

            color = "green" if pred_label == true_label_disp else "red"
            ax.set_title(f"{name}\nP:{pred_label}\nT:{true_label_disp}", 
                         fontsize=9, color=color)
    plt.tight_layout()
    plt.savefig("predictions_comparison.png")
    plt.close()

compare_predictions(
    [cnn_model, mobilenet_model, eff_model],
    ["CNN", "MobileNetV2", "EfficientNetB3"], 
    n=6
)


In [13]:
import zipfile
with zipfile.ZipFile("sugarcane_results.zip", "w") as zf:
    for f in os.listdir():
        if f.endswith(".png") or f.endswith(".txt") or f.endswith(".h5"):
            zf.write(f)

print("✅ All results and models saved in sugarcane_results.zip")


✅ All results and models saved in sugarcane_results.zip
